In [16]:
import torch
from torch import nn
import random
import gym
from gym import spaces
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [ ]:
class EnvNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(14, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 13),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x

In [ ]:
class ObservationRAM(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        # Observation space muss am Anfang festgelegt werden
        self.observation_space = spaces.Box(low=0, high=210, shape=(13, ))

    def observation(self, obs):
        # Die Daten die benutzt werden (y, score, cooldown, auto x pos)
        obs = obs[[14, 103, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117]].astype(float)

        # normalisieren
        obs[0] /= 176
        obs[2] /= 142
        for i in range(3, 13):
            obs[i] /= 160

        return obs

In [ ]:
def get_action_sample():
    # Random Actions wobei nach oben laufen am Wahrscheinlichsten ist
    x = random.randint(0, 101)
    if x < 90:
        return 1
    if x < 97:
        return 2
    return 0

def get_env_data(n):
    env_data = ObservationRAM(gym.make("ALE/Freeway-v5", obs_type="ram", render_mode="rgb_array", difficulty=1, mode=3))

    observation = env_data.reset()
    df = pd.DataFrame(observation).T
    # Actions: 0: nichts, 1: up, 2: down
    actions = []

    for i in range(n):
        # model bestimmt action
        action = get_action_sample()

        # observation 0 wird vor der schleife gespeichert
        if i != 0:
            df.loc[len(df)] = observation
        actions.append(action)
        observation, reward, done, info = env_data.step(action)
        if done:
            observation = env_data.reset()
    env_data.close()

    # df wird mit den richtigen Daten fertiggestellt
    df["actions"] = actions
    df = df.rename(columns={0: "y", 1: "score", 2: "cooldown",
                            3: "car1", 4: "car2", 5: "car3", 6: "car4", 7: "car5", 8: "car6", 9: "car7", 10: "car8", 11: "car9", 12: "car10"})

    return df

In [ ]:
def process_dfs(df):
    df = df.tail(len(df) - 1)

    dfY = df.copy()
    dfY.drop(["actions"], axis=1, inplace=True)

    dfY = dfY.drop(dfY.index[[0]])
    df = df.drop(df.index[[len(df) - 1]])
    dfY.index = df.index

    df = df.reset_index(drop=True)
    dfY = dfY.reset_index(drop=True)
    return df, dfY

In [22]:
def train(X, y, model, loss_fn, optimizer, batch_size):
    model.train()
    loss_sum = 0

    for i in range(round((len(y) / batch_size) + 0)):
        # In jedem durchlauf wird der ganze batch auf dem gpu gespeichert
        train_X = torch.from_numpy(X[i * batch_size:(i + 1) * batch_size]).cuda()
        train_y = torch.from_numpy(y[i * batch_size:(i + 1) * batch_size]).cuda()

        loss = 0

        # loss wird für jeden Datensatz berechnet
        for k in range(min(batch_size, len(train_X))):
            pred = model.forward(train_X[k].float())
            loss += loss_fn(pred.to(torch.float32), train_y[k].to(torch.float32))
        loss_sum += loss.item()

        # bp
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # return: avg loss
    return loss_sum / len(y)


def test(X, y, model, loss_fn, batch_size):
    loss_sum = 0

    model.eval()
    with torch.no_grad():
        for i in range(round((len(y) / batch_size) + 1)):
            # In jedem durchlauf wird der ganze batch auf dem gpu gespeichert
            test_X = torch.from_numpy(X[i * batch_size:(i + 1) * batch_size]).cuda()
            test_y = torch.from_numpy(y[i * batch_size:(i + 1) * batch_size]).cuda()

            # loss wird für jeden Datensatz berechnet
            for k in range(min(batch_size, len(test_X))):
                pred = model.forward(test_X[k].float())
                loss = loss_fn(pred, test_y[k])
                loss_sum += loss.item()

    loss_sum /= len(y)
    print(f"Avg loss: {loss_sum}!")
    return loss_sum

def train_test_model(X, y, batch_size, learning_rate):
    # X: input, y: target
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)

    # Reset parameters
    model = EnvNetwork().cuda()
    for layer in model.children():
        if hasattr(layer, "reset_parameters"):
            layer.reset_parameters()

    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    last_test_avg = 10000
    overfit = 0
    epochs = 100

    # Alle Dataframes zu numpy arrays
    train_X = train_X.to_numpy()
    test_X = test_X.to_numpy()
    train_y = train_y.to_numpy()
    test_y = test_y.to_numpy()

    for t in range(epochs):
        train_l = train(train_X, train_y, model, loss_fn, optimizer, batch_size)
        test_avg = test(test_X, test_y, model, loss_fn, batch_size)

        # Wenn der Test avg schlechter als vorher ist, wird der last_test_avg nicht aktualisiert
        # und der overfit counter erhöht. Falls der counter >= 10 wird das training frühzeitig abgebrochen
        if test_avg > last_test_avg:
            overfit += 1
        else:
            overfit = 0
            last_test_avg = test_avg
        if overfit >= 10:
            break
    return test_avg

In [23]:
def objective(trial):

    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.00001, 0.01),
        "batch_size": trial.suggest_int("batch_size", 16, 512)
    }

    loss = train_test_model(df, y, **params)
    return loss

In [24]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2023-01-05 22:23:33,838] A new study created in memory with name: no-name-f9324ba0-fc87-497b-9fae-ee5f88790f9f


Avg loss: 0.5636626430228825!
Avg loss: 0.0969713022196376!
Avg loss: 0.021625377405039056!
Avg loss: 0.015294704822394397!
Avg loss: 0.01314555542020537!
Avg loss: 0.011575227847336814!
Avg loss: 0.01027466408919354!
Avg loss: 0.008936749808431213!
Avg loss: 0.007759815481898551!
Avg loss: 0.006740521613189311!
Avg loss: 0.00529323974862979!
Avg loss: 0.004657006646682839!
Avg loss: 0.004267324886482623!
Avg loss: 0.0038495241014821335!
Avg loss: 0.0036769084704410273!
Avg loss: 0.0036057749783825364!
Avg loss: 0.0036189915817867223!
Avg loss: 0.003792641306171805!
Avg loss: 0.004181113681038873!
Avg loss: 0.0034818213999590598!
Avg loss: 0.0034012065062351327!
Avg loss: 0.0033753256154562513!
Avg loss: 0.0033987016030649175!
Avg loss: 0.003437273391015276!
Avg loss: 0.0034470888537070543!
Avg loss: 0.003462563784621314!
Avg loss: 0.003431373536860134!
Avg loss: 0.0034380102302457314!
Avg loss: 0.0036357670997363093!
Avg loss: 0.0034887336991336424!
Avg loss: 0.0033691557461674763!
Av

[I 2023-01-05 22:27:41,131] Trial 0 finished with value: 0.003144685754638996 and parameters: {'learning_rate': 0.006385014523276346, 'batch_size': 267}. Best is trial 0 with value: 0.003144685754638996.


Avg loss: 0.003144685754638996!
Avg loss: 0.597913809093455!
Avg loss: 0.5664134896948179!
Avg loss: 0.5624370819733284!
Avg loss: 0.5604665843000154!
Avg loss: 0.5580125170914483!
Avg loss: 0.5544011059049136!
Avg loss: 0.5533082914707901!
Avg loss: 0.5528536297700504!
Avg loss: 0.552585532239705!
Avg loss: 0.5523425529304323!
Avg loss: 0.5522337134453801!
Avg loss: 0.5521860463133296!
Avg loss: 0.5521336712307862!
Avg loss: 0.5521127803590584!
Avg loss: 0.5521045620046835!
Avg loss: 0.552058264077832!
Avg loss: 0.5520422456778344!
Avg loss: 0.5520417833631891!
Avg loss: 0.5520388588315919!
Avg loss: 0.5520176293151682!
Avg loss: 0.5520103206165918!
Avg loss: 0.5520591595596033!
Avg loss: 0.5520110737560286!
Avg loss: 0.5519999364764501!
Avg loss: 0.5520436121069713!
Avg loss: 0.5519616707509762!
Avg loss: 0.5519532756845519!
Avg loss: 0.5519475834968759!
Avg loss: 0.5519468312588999!
Avg loss: 0.5519109052971172!
Avg loss: 0.5519263670910273!
Avg loss: 0.5519260104873632!
Avg loss: 0

[I 2023-01-05 22:31:55,302] Trial 1 finished with value: 0.5518691974468577 and parameters: {'learning_rate': 0.007758108994574983, 'batch_size': 157}. Best is trial 0 with value: 0.003144685754638996.


...

Avg loss: 0.01599919926177853!
Avg loss: 0.27033690814170136!
Avg loss: 0.10091371328279886!
Avg loss: 0.05677331822027906!
Avg loss: 0.040963673626541776!
Avg loss: 0.03319759888790627!
Avg loss: 0.027849260474628428!
Avg loss: 0.024000545820555868!
Avg loss: 0.021384820407579823!
Avg loss: 0.01961595552146283!
Avg loss: 0.018439215662592507!
Avg loss: 0.01746668318765138!
Avg loss: 0.016622753433061212!
Avg loss: 0.015811605439756345!
Avg loss: 0.015095696866877287!
Avg loss: 0.01444765867782422!
Avg loss: 0.013882694551769426!
Avg loss: 0.013388726240620468!
Avg loss: 0.012969061754504732!
Avg loss: 0.012610424048341264!
Avg loss: 0.012304961671286547!
Avg loss: 0.012017202328687747!
Avg loss: 0.011784711401918781!
Avg loss: 0.011564569354430428!
Avg loss: 0.011367167771361484!
Avg loss: 0.011185011520003178!
Avg loss: 0.011012358096846194!
Avg loss: 0.010842465469926319!
Avg loss: 0.010680536540027786!
Avg loss: 0.010522394208782914!
Avg loss: 0.010368583879256365!
Avg loss: 0.0102

[I 2023-01-06 00:07:59,345] Trial 28 finished with value: 0.00743870951443024 and parameters: {'learning_rate': 0.0001818627374310395, 'batch_size': 230}. Best is trial 26 with value: 0.0021155083512607725.


Avg loss: 0.00743870951443024!
Avg loss: 0.06916191261658573!
Avg loss: 0.031631831810985514!
Avg loss: 0.020423128052126552!
Avg loss: 0.015737655833550152!
Avg loss: 0.0136270623361551!
Avg loss: 0.01241118761193831!
Avg loss: 0.011626235461755806!
Avg loss: 0.010992061979688196!
Avg loss: 0.010481765114323263!
Avg loss: 0.009986623300633667!
Avg loss: 0.009414184249933216!
Avg loss: 0.008870418954555271!
Avg loss: 0.008370568492704735!
Avg loss: 0.007897625193025398!
Avg loss: 0.00745523467677172!
Avg loss: 0.007037219048979798!
Avg loss: 0.006663461652430917!
Avg loss: 0.0063234191710251535!
Avg loss: 0.006015029969253038!
Avg loss: 0.005685056961798153!
Avg loss: 0.005331691574356125!
Avg loss: 0.004958253249913691!
Avg loss: 0.004552947279927011!
Avg loss: 0.004205914519100431!
Avg loss: 0.003918418592220735!
Avg loss: 0.003671807241371315!
Avg loss: 0.0034862889136706324!
Avg loss: 0.0033394647230804674!
Avg loss: 0.0032170173082458945!
Avg loss: 0.003116301821541391!
Avg loss: 

[I 2023-01-06 00:11:45,568] Trial 29 finished with value: 0.002342224313729295 and parameters: {'learning_rate': 0.001075924155665366, 'batch_size': 262}. Best is trial 26 with value: 0.0021155083512607725.


Avg loss: 0.002342224313729295!


In [27]:
study.trials_dataframe()
study.best_trial.params

{'learning_rate': 0.0014335761400056032, 'batch_size': 234}